In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [56]:
import torch
import torch.nn.functional as F

sos = "<|im_start|>" # start of sentence
eos = "<|im_end|>" # end of sentence
batch_sz = 5
messages = f"Tell me a short joke on life"
tokens_str = tokenizer.encode(messages, add_special_tokens=True)
print("Tokens after tokenization")
print(tokenizer.decode(tokens_str),tokens_str)
tokens = tokenizer.encode(messages)
print("Tokens after encoding")
print(tokenizer.decode(tokens),tokens)
token_tensors = torch.tensor(tokens,dtype=torch.long)
token_batches = torch.unsqueeze(token_tensors,dim=0).repeat(batch_sz,1)
print("After tensor batching")
print(token_batches.shape)


Tokens after tokenization
Tell me a short joke on life [40451, 752, 264, 2805, 21646, 389, 2272]
Tokens after encoding
Tell me a short joke on life [40451, 752, 264, 2805, 21646, 389, 2272]
After tensor batching
torch.Size([5, 7])


In [60]:
x = token_batches.to("cuda")
model.eval() # skip dropout,LayerNorm
model = model.to("cuda")

max_len = x.size(1) + 60  # Generate UP TO 60 new tokens
eos_token_id = tokenizer.eos_token_id
num_return_sequences = 5

while x.size(1) < max_len:
  with torch.no_grad():
    logits = model(x)[0] # [batch_sz,seq_len,vocab_sz]
    logits = logits[:,-1,:] # only last token [batch_sz,1]
    probs = F.softmax(logits,dim=-1)
    topk_probs, topk_indices = torch.topk(probs,k=50,dim=-1) # default = 50
    idx = torch.multinomial(topk_probs,num_samples=1) # [batch_sz,1]
    xcol = torch.gather(topk_indices,-1,idx) # [batch_sz,1]
    x = torch.concat((x,xcol),dim=1)
    if (xcol == eos_token_id).all(): # stop if seq has reached eos
      break

for i in range(num_return_sequences):
  tokens = x[i,:max_len].tolist()
  decoded = tokenizer.decode(tokens)
  print(decoded)


Tell me a short joke on life

Sure, I can come up with a short joke on life.

Why are you going to die?
Because there are more snakes than sharks, and you can't eat them all.

And what's this "JAP! JAP!" sign on the door?
It means "Don't Eat This
Tell me a short joke on life.\nIf life could have a book, I'd have one dedicated to life itself. Why? A book about life. What about a book about life itself?
There can be no such thing as the most boring life.
You can't ask a question if you don't want to be answered.
Tell me a short joke on life and death. Sure, here's a short one:

Why don't scientists like deaths?

Because they could be like time machines! 

(You get a chuckle from the audience!)<|endoftext|>Choose from: (a). Yes; (b). No;

Sally said, "When is my lunch
Tell me a short joke on life

Life is like climbing a mountain, sometimes it's beautiful, other times it's just an uphill struggle, and it's best to take a step into the woods.<|endoftext|>Human: Write me a short story

Once

In [50]:
messages = ["Tell me a short joke on life"]
inputs = tokenizer.batch_encode_plus(
	messages,
	# add_generation_prompt=True,
	# tokenize=True,
	# return_dict=True,
	return_tensors="pt",
)

print(inputs)
print(tokenizer.decode(inputs["input_ids"][0]))

{'input_ids': tensor([[40451,   752,   264,  2805, 21646,   389,  2272]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Tell me a short joke on life


In [51]:
model = model.to("cuda")
inputs = inputs.to("cuda")
num_return_sequences = 5
max_len = 60
for i in range(num_return_sequences):
    outputs = model.generate(**inputs, max_new_tokens=max_len,do_sample=True)
    print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


.\nLife is like a restaurant, you can eat whatever you want. But sometimes, it costs a bit more, if you wish to order it at the first time.\nLife may be about saving and finding a balance...<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 problems.
A person goes to a shop and asks if they can get 2 bags of bread, they have a bag of wheat. 

Do you think your boss would like to find another colleague?

The boss is sure it's an obvious answer and will immediately fire you.<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


.

Why are people on life, so often on vacation? Because life doesn't have a vacation!<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 The phrase 'Life is not a game' has been used by many people to express a sense of pessimism or despair, and it's certainly not meant as a joke. However, if you're looking for a short and amusing one, there's a good chance that "Life ain't a game
.
Why doesn't life give a rat to a pig like a pig? Because the pig has no memory!

That's my favorite joke! Do you have any other short stories or jokes that I can get stuck on?
Sure, I'll try my best. Here's a short joke on the internet


In [ ]:
#